# Explain Attacking BERT models using CAptum

Captum is a PyTorch library to explain neural networks
Here we show a minimal example using Captum to explain BERT models from TextAttack

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

In [1]:
import torch
from copy import deepcopy

In [2]:
from textattack.datasets import HuggingFaceDataset
from textattack.models.tokenizers import AutoTokenizer
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
from captum.attr import visualization as viz

In [4]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(device) 

cuda:2


In [5]:
dataset = HuggingFaceDataset("ag_news", None, "train")
original_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
original_tokenizer = AutoTokenizer("textattack/bert-base-uncased-ag-news")
model = HuggingFaceModelWrapper(original_model,original_tokenizer)

Using custom data configuration default
Reusing dataset ag_news (/u/ss7mu/.cache/huggingface/datasets/ag_news/default/0.0.0/fb5c5e74a110037311ef5e904583ce9f8b9fbc1354290f97b4929f01b3f48b1a)
textattack: Loading datasets dataset ag_news, split train.


In [42]:
def captum_form(encoded):
    input_dict = {k: [_dict[k] for _dict in encoded] for k in encoded[0]}
    batch_encoded = { k: torch.tensor(v).to(device) for k, v in input_dict.items()}
    return batch_encoded

def get_text(tokenizer,input_ids,token_type_ids,attention_mask):
    list_of_text = []
    number = input_ids.size()[0]
    for i in range(number):
        ii = input_ids[i,].cpu().numpy()
        tt = token_type_ids[i,]
        am = attention_mask[i,]
        txt = tokenizer.decode(ii, skip_special_tokens=True)
        list_of_text.append(txt)
    return list_of_text
    
sel =2
encoded = model.tokenizer.batch_encode([dataset[i][0]['text'] for i in range(sel)])
labels = [dataset[i][1] for i in range(sel)]

batch_encoded = captum_form(encoded)

clone = deepcopy(model)
clone.model.to(device)

def calculate(input_ids,token_type_ids,attention_mask):
    #convert back to list of text
    return clone.model(input_ids,token_type_ids,attention_mask)[0]
    
# x = calculate(**batch_encoded)    

lig = LayerIntegratedGradients(calculate, clone.model.bert.embeddings)
# lig = InternalInfluence(calculate, clone.model.bert.embeddings)
# lig = LayerGradientXActivation(calculate, clone.model.bert.embeddings)

bsl = torch.zeros(batch_encoded['input_ids'].size()).type(torch.LongTensor).to(device)
labels = torch.tensor(labels).to(device)

attributions,delta = lig.attribute(inputs=batch_encoded['input_ids'],
                              baselines=bsl,
                              additional_forward_args=(batch_encoded['token_type_ids'], batch_encoded['attention_mask']),
                              n_steps = 10,
                              target = labels,
                              return_convergence_delta=True
                              )
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

In [53]:
# print(attributions.size())
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

In [54]:
from textattack.attack_recipes import PWWSRen2019
attack = PWWSRen2019.build(model)

textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [58]:
results_iterable = attack.attack_dataset(dataset, indices=range(10))

viz_list = []

for n,result in enumerate(results_iterable):
    orig = result.original_text()
    pert = result.perturbed_text()
    
    encoded = model.tokenizer.batch_encode([orig])
    batch_encoded = captum_form(encoded)
    x = calculate(**batch_encoded) 
    print(x)
    print(dataset[n][1])
    
    pert_encoded = model.tokenizer.batch_encode([pert])
    pert_batch_encoded = captum_form(pert_encoded)
    x_pert = calculate(**pert_batch_encoded) 


    attributions,delta = lig.attribute(inputs=batch_encoded['input_ids'],
#                               baselines=bsl,
                              additional_forward_args=(batch_encoded['token_type_ids'], batch_encoded['attention_mask']),
                              n_steps = 10,
                              target = torch.argmax(calculate(**batch_encoded)).item(),
                              return_convergence_delta=True
                              )
    attributions_pert,delta_pert = lig.attribute(inputs=pert_batch_encoded['input_ids'],
#                               baselines=bsl,
                              additional_forward_args=(pert_batch_encoded['token_type_ids'], pert_batch_encoded['attention_mask']),
                              n_steps = 10,
                              target = torch.argmax(calculate(**pert_batch_encoded)).item(),
                              return_convergence_delta=True
                              )
    
    orig = original_tokenizer.tokenizer.tokenize(orig)
    pert = original_tokenizer.tokenizer.tokenize(pert)
    
    atts = attributions.sum(dim=-1).squeeze(0)
    atts = atts / torch.norm(atts)

    
    
    atts_pert = attributions_pert.sum(dim=-1).squeeze(0)
    atts_pert = atts_pert / torch.norm(atts)
    
    
    
    all_tokens = original_tokenizer.tokenizer.convert_ids_to_tokens(batch_encoded['input_ids'][0])
    all_tokens_pert = original_tokenizer.tokenizer.convert_ids_to_tokens(pert_batch_encoded['input_ids'][0])
    
        
    v = viz.VisualizationDataRecord(
                    atts[:45].detach().cpu(),
                    torch.max(x).item(),
                    torch.argmax(x,dim=1).item(),
                    dataset[n][1],
                    2,
                    atts.sum().detach(), 
                    all_tokens[:45],
                    delta)
    
    v_pert = viz.VisualizationDataRecord(
                    atts_pert[:45].detach().cpu(),
                    torch.max(x_pert).item(),
                    torch.argmax(x_pert,dim=1).item(),
                    dataset[n][1],
                    2,
                    atts_pert.sum().detach(), 
                    all_tokens_pert[:45],
                    delta_pert)
    viz_list.append(v)
    viz_list.append(v_pert)

#     print(result.perturbed_text())
    print(result.__str__(color_method='ansi'))



tensor([[-3.3016, -2.1467,  2.0953,  2.7272]], device='cuda:2',
       grad_fn=<AddmmBackward>)
2
Business (96%) --> Sci/tech (68%)

Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

Wall St. suffer Claw Back Into the lightlessness (Reuters) Reuters - Short-sellers, Wall Street's dwindle\isthmus of ultra-cynics, are examine greenish again.
tensor([[-4.8619, -0.8199,  3.9263,  1.1104]], device='cuda:2',
       grad_fn=<AddmmBackward>)
2
Business (100%) --> Sci/tech (50%)

Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.

Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investiture firm Carlyle Group,\which has a reputation for ca-ca well-timed and occasional

In [60]:
print('\033[1m', 'Visualizations For AG NEWS', '\033[0m')
viz.visualize_text(viz_list)

 Visualizations For AG NEWS 


In [ ]:
# reference for viz datarecord
# def __init__(
#         self,
#         word_attributions,
#         pred_prob,
#         pred_class,
#         true_class,
#         attr_class,
#         attr_score,
#         raw_input,
#         convergence_score,
#     ):